# Cuaderno RETO 1: Exploración de los datos

En este cuaderno se realiza la **exploración inicial del dataset locales** proporcionado
para la práctica de Bases de Datos NoSQL.

El objetivo de este reto es **comprender la estructura, el contenido y la calidad de los datos**
sin realizar todavía tareas de limpieza ni inserción en la base de datos.

Esta exploración permitirá identificar:
- Número de registros y atributos
- Tipos de datos
- Valores nulos
- Posibles claves y relaciones
- Problemas de calidad de los datos


### Contexto del conjunto de datos (Locales)

El conjunto de datos describe los **locales comerciales registrados en el Censo de Locales
y Actividades del Ayuntamiento de Madrid**, incluyendo información administrativa,
territorial, geográfica y de clasificación de la actividad económica.

Cada registro identifica un local mediante un código único (`id_local`) y lo asocia a una
dirección oficial del edificio, así como a su distrito, barrio y sección censal.
Los locales se clasifican según su tipo de acceso (puerta de calle, agrupado,
puerta de calle asociado) y su situación administrativa (abierto, cerrado, en obras,
baja o uso vivienda).

El dataset incluye información detallada sobre la localización del local, tanto a nivel
de dirección postal como mediante coordenadas UTM cuando procede, así como datos
relativos a agrupaciones de locales (por ejemplo, mercados o centros comerciales).

Asimismo, el fichero constituye la base común para la información de actividad económica,
ya que un mismo local puede tener asociadas una o varias actividades en otros ficheros
del conjunto de datos.

Los datos reflejan la última situación administrativa conocida del local en el momento
de la extracción.


In [ ]:
# Imports necesarios
import pandas as pd 

# Configuración para mostrar todas las filas y columnas sin truncar
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)

### Carga del dataset

En primer lugar se carga el fichero CSV.
Este dataset contiene datos relativos a las terrazas de hostelería y restauración 
que constan asociadas a locales del censo de locales y actividades del Ayuntamiento de 
Madrid.

El fichero utiliza el carácter `;` como separador de campos.

In [ ]:
df_locales = pd.read_csv("../../data/raw/locales202312.csv", sep=';')

print(df_locales.shape)

El resultado anterior indica que en total tenemos 151162 filas y 48 columnas presentes en el dataset.

A continuación vamos a visualizar las primeras filas del dataset para obtener una primera idea del contenido y formato de los datos.

In [ ]:
df_locales.head()

A continuación se muestra una muestra aleatoria de registros.
Esto permite observar valores representativos que no necesariamente
aparecen en las primeras filas del fichero.

In [ ]:
df_locales.sample(5, random_state=42)

### Estructura y tipos de datos

Se analiza la estructura del DataFrame para identificar:
- Nombre de los atributos
- Tipo de dato asignado por pandas
- Número de valores no nulos por columna

In [ ]:
df_locales.info()

A partir de esta información se puede observar que:
- Existen 4 columnas de tipo float. 
- Existen 20 columnas de tipo int64. 
- Existen 24 columnas de tipo str. 
- Los campos que representan fechas están representados como str.

### Análisis de valores nulos

Se comprueba la existencia de valores nulos en el dataset.
En esta fase **no se eliminan ni corrigen**, únicamente se identifican.

In [ ]:
df_locales.isnull().sum()[df_locales.isnull().sum() > 0]

El resultado muestra qué columnas contienen valores nulos y cuántos registros se ven afectados en cada caso.

### Análisis de columnas de texto

Se identifican las columnas de tipo texto para analizar posibles problemas de calidad, como espacios en blanco innecesarios o formatos inconsistentes.

In [ ]:
text_cols = df_locales.select_dtypes(include="object").columns
print(text_cols)

A continuación se inspeccionan los valores únicos de una columna de texto representativa para observar posibles inconsistencias.

In [ ]:
df_locales["desc_barrio_local"].unique()

En los valores mostrados se puede observar la presencia de espacios en blanco
al final de las cadenas de texto, lo que indica que será necesario realizar
un preprocesado mínimo antes de la inserción en la base de datos.

### Identificación de columnas de fecha

Se identifican las columnas que contienen información temporal.
Estas columnas suelen comenzar por el prefijo `fx_`.

In [ ]:
date_cols = [c for c in df_locales.columns if "fx_" in c.lower()]
print(date_cols)

Las columnas identificadas contienen fechas almacenadas como texto,
por lo que será necesario convertirlas a un formato de fecha adecuado
en fases posteriores para permitir consultas temporales.

### Identificación de columnas de hora

Se identifican las columnas que contienen información temporal.
Estas columnas suelen comenzar por el prefijo `hora`.

In [ ]:
hour_cols = [c for c in df_locales.columns if "hora" in c.lower()]
print(hour_cols)

Las columnas identificadas contienen horas almacenadas como texto,
por lo que será necesario convertirlas a un formato de hora adecuado
en fases posteriores para permitir consultas temporales.

### Listado completo de columnas

A continuación, se muestra el listado completo de atributos del dataset para identificar posibles claves y relaciones con otros ficheros.

In [ ]:
# Crea una tabla con dos columnas: "Columna" y "Tipo"
tipos_df = df_locales.dtypes.reset_index()
tipos_df.columns = ['Columna', 'Tipo de Dato']

tipos_df.style.set_properties(**{'text-align': 'left'}).hide(axis='index')

### Conclusión del dataset y campos relevantes para el modelo de datos

A partir de la exploración realizada, se identifican aquellos campos que
resultan especialmente relevantes para la construcción del modelo de datos,
ya sea por su papel como identificadores, por permitir relaciones con otros
datasets o por aportar información estructural clave.


In [ ]:
cols_locales_clave = [
    # Identificación principal
    "id_local",

    # Contexto territorial
    "id_distrito_local",
    "desc_distrito_local",
    "id_barrio_local",
    "desc_barrio_local",
    "id_seccion_censal_local",

    # Clasificación administrativa
    "id_tipo_acceso_local",
    "desc_tipo_acceso_local",
    "id_situacion_local",
    "desc_situacion_local",

    # Dirección del edificio
    "id_ndp_edificio",
    "id_vial_edificio",
    "clase_vial_edificio",
    "desc_vial_edificio",
    "num_edificio",
    "cal_edificio",
    "secuencial_local_PC",

    # Dirección de acceso al local
    "id_ndp_acceso",
    "id_vial_acceso",
    "clase_vial_acceso",
    "desc_vial_acceso",
    "num_acceso",
    "cal_acceso",

    # Agrupaciones
    "id_agrupacion",
    "nombre_agrupacion",
    "id_tipo_agrup",
    "desc_tipo_agrup",
    "id_planta_agrupado",
    "id_local_agrupado",

    # Identificación comercial
    "rotulo",

    # Información temporal
    "fx_datos_ini",
    "fx_datos_fin"
]

df_locales[cols_locales_clave].drop_duplicates().sample(10, random_state=42)

Los campos seleccionados del dataset permiten identificar de forma
única cada establecimiento mediante el identificador `id_local`, que actúa como
elemento central de relación con el resto de datasets analizados (terrazas,
licencias y actividades económicas).

Se incluyen los atributos territoriales (`id_distrito_local`, `id_barrio_local`,
`id_seccion_censal_local` y sus descripciones) con el objetivo de permitir
análisis y agregaciones geográficas, así como para facilitar la comprensión del
contexto administrativo del local.

Asimismo, se consideran los campos de clasificación administrativa del local,
como el tipo de acceso y su situación (`id_tipo_acceso_local`,
`id_situacion_local`), dado que condicionan la existencia de terrazas, licencias
y actividades asociadas.

La información de dirección, tanto del edificio como del acceso real al local,
se incorpora para reflejar la complejidad del modelo de localización, especialmente
en casos de locales agrupados o con accesos diferenciados.

Por último, se incluyen los campos relacionados con agrupaciones de locales y
la identificación comercial (`rotulo`), ya que aportan información estructural
relevante para el diseño posterior del modelo de datos y para la definición de
posibles documentos embebidos o relaciones entre colecciones.

Al igual que en el resto de datasets, esta selección tiene un carácter
exploratorio y no implica que todos los campos deban mantenerse en el modelo
final, sino que sirve como base para evaluar relaciones, redundancias y
decisiones de modelado en fases posteriores.
